In [2]:
import random
import numpy as np
from fuzzywuzzy import fuzz
import time

In [3]:
from random import shuffle

class Forest:
    def BuildMinHash(self, vocabSize):
        hashes = []
        for _ in range(self.nbits):
            hashList = [i for i in range(1, vocabSize+1)]
            random.shuffle(hashList)
            hashes.append(hashList)
        return hashes
    
    def Shingle(self, text, k):
        return set([text[i:i+k] for i in range(len(text)-k+1)])

    # Generates a one-hot binary signature of a given set of shingles
    def GenSparseSig(self, shingledWord):
        sig = [0]*len(self.vocab)
        for shingle in shingledWord:
            try:
                sig[self.shingleVocabIndex[shingle]] = 1
            except KeyError:
                continue
                
        return sig
    
    # MinHash a vector using the pre-computed minhash functions
    def MinHash(self, vector):
        signature = []
        for func in self.minHashFunctions:
            for i in range(1, len(self.vocab)+1):
                #Funcs used as random, non-repeating, index ordering instead of checking index one by one
                idx = func[i-1]-1
                
                # Find first index in each hash function that returns 1  
                if vector[idx] == 1:
                    signature.append(idx)
                    break
        return signature 
                    
    def BandSignature(self, signature):
        assert len(signature) % self.b == 0
        
        bandLength = len(signature)//self.b
        bands = []
        for i in range(0, len(signature), bandLength):
            bands.append(signature[i:i+bandLength])

        return bands
    
    def WordToSignature(self, word):
        wordShingles = self.Shingle(word, self.shingleSize)
        oneHotSig = self.GenSparseSig(wordShingles)
        return self.MinHash(oneHotSig)
    
    def GetSimilarWords(self, word):
        signature = self.WordToSignature(word)
        bands = self.BandSignature(signature)
        candidateWords = set()

        for band in bands:
            bucketKey = ",".join([str(num) for num in band])
            try:
                candidateWords = candidateWords.union(forest.buckets[bucketKey])
            except KeyError:
                continue
        
        return candidateWords
        
    
    def BuildForest(self):

        print("Shingling Words")
        # Shingle starting words and create vocab set
        wordShingleSets = []
        for word in self.wordList:
            wordShingles = self.Shingle(word, self.shingleSize)
            
            self.vocab = self.vocab.union(wordShingles)
            wordShingleSets.append(wordShingles)
        
        #Creating an hash of the location of each shingle in vocab will speedup hashing time
        self.shingleVocabIndex = {}
        self.vocab = list(self.vocab)
        for i, shingle in enumerate(self.vocab):
            self.shingleVocabIndex[shingle] = i
        
        # Generate and save hashfunction to later add new words to forest
        self.minHashFunctions = self.BuildMinHash(len(self.vocab))
        self.wordSignatures = {}

        print("Hashing Words")
        # Create a signature for each word using min-hash
        for word, wordShingles in zip(wordList, wordShingleSets):
            oneHotSig = self.GenSparseSig(wordShingles)
            denseSig = self.MinHash(oneHotSig)
            self.wordSignatures[word] = denseSig

        
        print("Banding Words")
        # Band words together into buckets (LSH)
        self.buckets = {}
        for word, signature in self.wordSignatures.items():   
            for band in self.BandSignature(signature):
                bucketKey = ",".join([str(num) for num in band])

                if bucketKey in self.buckets:
                    self.buckets[bucketKey].add(word)
                else:
                    self.buckets[bucketKey] = set([word])

    def __init__(self, words, shingleSize=2, nbits=20, b=5):
        self.wordList = words
        self.shingleSize = shingleSize
        self.nbits = nbits
        self.b = b
        self.vocab = set()
        
        self.BuildForest()

In [11]:
print("Reading Words")
with open("Words.txt", 'r') as raw:
    wordList = [word.strip() for word in raw if len(word.strip()) > 5]

Reading Words


In [17]:
def SaltText(text, saltCharacters):
    newText = text
    alphabet = "0 1 2 3 4 5 6 7 8 9 , . | ] [ - _ = + / ! @ # $ % ^ & * ) (".split(' ')
    
    for _ in range(saltCharacters):
        randomI = random.randint(0, len(newText))
        newText = newText[:randomI] + random.choice(alphabet) + newText[randomI:]
         
    return newText

In [18]:
saltedWords = [SaltText(word, len(word)//3) for word in wordList]
shuffledSaltedWords = saltedWords[:]
random.shuffle(shuffledSaltedWords)

In [143]:
#Naive Method
correctMatches = 0

# Compares each word in word-list to each word in salted-word-list to find best match
# O(n^2)
start_time = time.time()
for word, saltedWord in zip(wordList, saltedWords):
    #print(word)
    bestMatch = ""
    bestScore = 0

    for saltWord in shuffledSaltedWords:
        score = fuzz.ratio(word, saltWord)
        if score > bestScore:
            bestScore = score
            bestMatch = saltWord
    
    if bestMatch == saltedWord:
        correctMatches += 1
        
print("Runtime:", time.time() - start_time)
print(f"Accuracy: {correctMatches}/{len(wordList)}")

Runtime: 472.75535225868225
Accuracy: 9862/9974


In [21]:
#Using LSH
incorrectMatches = []
correctMatches = []
start_time = time.time()

#Build Forest
forest = Forest(wordList, shingleSize=2, nbits=20, b=10)
print("Time to Build Forest:", time.time() - start_time)


#Only compare words the same buckets
for word, saltedWord in zip(wordList, saltedWords):
    bestMatch = ""
    bestScore = 0
    
    for candidateWord in forest.GetSimilarWords(saltedWord):
        score = fuzz.ratio(saltedWord, candidateWord)
        
        if score > bestScore:
            bestScore = score
            bestMatch = candidateWord
    
    if bestMatch == word:
        correctMatches.append((word, saltedWord, bestMatch))
    else:
        incorrectMatches.append((word, saltedWord, bestMatch))
        
        




print("Runtime:", time.time() - start_time)
print(f"Accuracy: {len(correctMatches)}/{len(wordList)}")

Shingling Words
Hashing Words
Banding Words
Time to Build Forest: 1.8464140892028809
Runtime: 11.210526704788208
Accuracy: 6387/6394


In [23]:
incorrectMatches

[('libraries', 'l=ib^r[aries', 'salaries'),
 ('mountains', 'moun7ta$i[ns', 'mounting'),
 ('police', 'p=ol]ice', 'voices'),
 ('polish', 'p+o4lish', 'publish'),
 ('spears', 's4pear@s', 'speaks'),
 ('update', 'up/d*ate', 'donate'),
 ('urgent', 'urg%e-nt', 'current')]